In [42]:
# Ram Ram

In [43]:
import networkx as nx
import random

In [44]:
commodities=['P1','P2','P3','P4']
total_nodes=['A','B','C','D','E','F','G']
depot_nodes=['A','B']
customer_nodes=['C','D','E','F','G']

vehicle_content=0
with open(r"C:\Users\Acer Pc\Desktop\VRP OpperHeimer\OpperHeimer Code\OpperHeimer Code\Data\vessel.txt") as f:
    vehicle_content=f.read()
vehicle_content=vehicle_content.split("\n")
if vehicle_content[-1] =='' or vehicle_content[-1] =='\n':
    vehicle_content=vehicle_content[:-1]
vehicle_dict={}
for item in vehicle_content:
    item=item.split(":")
    vehicle_cap=int(item[1].split(",")[0].split("(")[1].strip())
    vehicle_cost=int(item[1].split(",")[1].split(")")[0].strip())
    vehicle_dict[item[0].strip()]=[vehicle_cap,vehicle_cost]

capacity_content=0
with open(r"C:\Users\Acer Pc\Desktop\VRP OpperHeimer\OpperHeimer Code\OpperHeimer Code\Data\capacity.txt") as f:
    capacity_content=f.read()
capacity_content=capacity_content.split("\n")

if capacity_content[-1]==" " or capacity_content[-1]=="\n":
    capacity_content=capacity_content[:-1]

capacity_dict={}
for item in capacity_content:
    item=item.split(":")
    depot_id=item[0].split(",")[0].split("(")[1].strip()
    product_id=item[0].split(",")[1].split(")")[0].strip()
    depot_cap=int(item[1].strip())
    capacity_dict[(depot_id,product_id)]=depot_cap

time_content=0
with open(r"C:\Users\Acer Pc\Desktop\VRP OpperHeimer\OpperHeimer Code\OpperHeimer Code\Data\time.txt") as f:
    time_content=f.read()
time_content=time_content.split("\n")

if time_content[-1]==" " or time_content[-1]=="\n":
    time_content=time_content[:-1]
mapper={0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G'}
time_dict={}
for item in time_content:
    item=item.split(":")
    source_depot=int(item[0].split(",")[0].split("(")[1].strip())
    destination_depot=int(item[0].split(",")[1].split(")")[0].strip())
    time_to_travel=int(item[1].strip())
    time_dict[(mapper[source_depot],mapper[destination_depot])]=time_to_travel
    time_dict[(mapper[destination_depot],mapper[source_depot])]=time_to_travel

# demand_content=0
# with open(r"C:\Users\Acer Pc\Desktop\VRP OpperHeimer\OpperHeimer Code\OpperHeimer Code\Data\demand.txt") as f:
#     demand_content=f.read()
# demand_content=demand_content.split("\n")

# if demand_content[-1]==" " or demand_content[-1]=="\n":
#     demand_content=demand_content[:-1]

with open(r"C:\Users\Acer Pc\Desktop\VRP OpperHeimer\OpperHeimer Code\OpperHeimer Code\Data\demand_by_day.txt","r") as f:
    x=f.read()

demand_content=x.split("\n")
demand_dict={}
for item in demand_content:
    if len(item)!=0:
        item=item.split(":")
        tup_list=item[0].strip().split(",")
        actual_node=tup_list[0].split("(")[1].strip().split("'")[1]
        actual_product=tup_list[1].strip().split("'")[1]
        actual_day=tup_list[2].split(")")[0].strip().split("'")[1]
        actual_demand=int(item[1].strip())
        demand_dict[(actual_node,actual_product,actual_day)]=actual_demand


days=[]
for i in range(1,16):
    days.append('D'+str(i))

In [45]:


# demand_low_param=10000
# demand_up_param=15000

# demand_dict={}

# for node in total_nodes:
#     for c in range(len(commodities)):
#         for d in range(len(days)):
#             if node in depot_nodes:
#                 demand_dict[(node,commodities[c],days[d])]=0
#             else:
#                 demand_dict[(node,commodities[c],days[d])]=random.randint(demand_low_param,demand_up_param)



In [46]:
G=nx.Graph()
for node in total_nodes:
    G.add_node(node)
for edge in time_dict.keys():
    if edge[0]!=edge[1]:
        G.add_edge(edge[0],edge[1],weight=time_dict[edge])


In [47]:
from itertools import combinations,permutations

global_paths_set=set()
all_paths_list=[]
roundtrips_dict={}

for depot in depot_nodes:
    roundtrips_dict[depot]=set()
    for node in total_nodes:
        for path in list(nx.all_simple_paths(G,depot,node)):
            if depot not in path[1:]:
                temp=path.copy()
                temp.append(depot)
                temp=tuple(temp)
                roundtrips_dict[depot].add((temp))

# for permutation in permutations(depot_nodes, 2):
#     source_depot=permutation[0]
#     destination_depot=permutation[1]
#     for path in list(nx.all_simple_paths(G,source=source_depot,target=destination_depot)):
#         temp=tuple(path)
#         all_paths_list.append(path)
#         global_paths_set.add(temp)

for depot in depot_nodes:
    for path in roundtrips_dict[depot]:
        global_paths_set.add(path)



        
        

In [48]:
# global_paths_set_copy=global_paths_set.copy()

# !!!!!!!!!!!!!!!!!!!! *************** WARNING WE HAVE CHANGED THE GLOBAL PATHS SET INTO A LIST FOR CONVENIENCE PURPOSES
# AS PYTHON WONT ALLOW SUBSCRIPTING A SET VIA POSITION INDEX

global_paths_set=list(global_paths_set)

path_dict={}
for keys,values in vehicle_dict.items():
    for c in range(len(commodities)):
        for d in range(len(days)):
            path_dict[(keys,commodities[c],days[d])]={}
            for path_index in range(len(global_paths_set)):
                temp={'CustomerSet':set(),'Start':-1,'End':-1,'DepotSet':set(),'Path':[],'TotalCustomerDemand':0,'VehicleCapacity':0,'EachCustomerDemand':[],'Day':days[d],'Commodity':commodities[c],'VehicleType':keys,'MaxCustomerDemand':-1}

                #Setting start and end
                temp['Start']=global_paths_set[path_index][0]
                temp['End']=global_paths_set[path_index][-1]

                temp['Path']=global_paths_set[path_index]
                temp['NodeSet']=[]
                #Customer Sets and Depot Sets
                temp_customer_demand=0
                max_customer_demand=0
                for i in range(len(global_paths_set[path_index])):
                    if global_paths_set[path_index][i] not in temp['NodeSet']:
                        temp['NodeSet'].append(global_paths_set[path_index][i])
                    if global_paths_set[path_index][i] in customer_nodes:
                        temp['CustomerSet'].add(global_paths_set[path_index][i])
                        temp_customer_demand+=(demand_dict[(global_paths_set[path_index][i],commodities[c],days[d])])
                        max_customer_demand=max(max_customer_demand,demand_dict[(global_paths_set[path_index][i],commodities[c],days[d])])
                        temp['MaxCustomerDemand']=max_customer_demand
                        #print(temp['MaxCustomerDemand'])
                    elif global_paths_set[path_index][i] in depot_nodes:
                        temp['DepotSet'].add(global_paths_set[path_index][i])
                    temp['EachCustomerDemand'].append(demand_dict[(global_paths_set[path_index][i],commodities[c],days[d])])
                
                temp['TotalCustomerDemand']=temp_customer_demand
                temp['VehicleCapacity']=values[0]
                max_serving_capacity=0
                for node in temp['DepotSet']:
                    max_serving_capacity+=capacity_dict[(node,commodities[c])]
                if temp['VehicleCapacity'] >= temp['MaxCustomerDemand'] and max_serving_capacity>=temp['MaxCustomerDemand']:
                    temp['Feasible']= True
                else:
                    temp['Feasible']=False
                
                temp['Cost']=0
                #Change to TSP Costs.
                # sorted_node_tuple=tuple(sorted(list(temp['NodeSet'])))
                # temp['TSPRoute']=global_paths_set_tsp_mapped[sorted_node_tuple]
                # temp['TSPNodeSet']=[]
                # temp['TSPFeasible']=False
                # temp['RouteNo']=global_paths_set_tsp_indices[sorted_node_tuple]
                # for node_tsp in temp['TSPRoute']:
                #     if node_tsp not in temp['TSPNodeSet'] and node_tsp in temp['Path']:
                #         temp['TSPNodeSet'].append(node_tsp)
                # if temp['TSPNodeSet']==temp['NodeSet']:
                    #print("Yay! It seems we can go via a low cost way")
                    # tsp_cost=0
                    # normal_cost=0
                    # for i in range(len(temp['TSPRoute'])-1):
                    #     tsp_cost+=time_dict[(temp['TSPRoute'][i],temp['TSPRoute'][i+1])]*vehicle_dict[keys][1]
                    # for i in range(len(global_paths_set[path_index])-1):
                    #     normal_cost+=time_dict[(global_paths_set[path_index][i],global_paths_set[path_index][i+1])]*vehicle_dict[keys][1]
                    
                    # temp['Cost']=min(tsp_cost,normal_cost)
                    # if tsp_cost<=normal_cost:
                    #     temp['TSPFeasible']=True                  
                # else:
                    # print("Nope ! Unable")
                for i in range(len(global_paths_set[path_index])-1):
                    temp['Cost']+=time_dict[(global_paths_set[path_index][i],global_paths_set[path_index][i+1])]*vehicle_dict[keys][1]
                
                temp['ConsumptionTracker']=0

                
                temp['CustomerSet']=list(temp['CustomerSet'])
                temp['DepotSet']=list(temp['DepotSet'])
                if temp['Feasible']==True:
                    #temp['ConsumptionTracker']=temp['TotalCustomerDemand']
                    path_dict[(keys,commodities[c],days[d])][path_index]=temp




In [49]:
feasible_path_dict={}
for keys,values_vehicle in vehicle_dict.items():
    for c in range(len(commodities)):
        for d in range(len(days)):
            feasible_path_dict[(keys,commodities[c],days[d])] = {}
            for path_index,values in path_dict[(keys,commodities[c],days[d])].items():
                temp=[]
                consumption_temp={}
                if len(path_dict[(keys,commodities[c],days[d])][path_index]['Path'])>=3:
                    supply_temp=0
                    # demand_temp=0
                    checker=True
                    for node_index in range(len(path_dict[(keys,commodities[c],days[d])][path_index]['Path'])-1):
                        if checker==True:
                            node=path_dict[(keys,commodities[c],days[d])][path_index]['Path'][node_index]
                            rem_demand=0
                            for rem_index in range(node_index+1, len(path_dict[(keys,commodities[c],days[d])][path_index]['Path'])-1):
                                new_visited_node=path_dict[(keys,commodities[c],days[d])][path_index]['Path'][rem_index]
                                if new_visited_node in customer_nodes:
                                    rem_demand+=demand_dict[(new_visited_node,commodities[c],days[d])]

                            if node in depot_nodes and node_index!=0:
                                if supply_temp==values_vehicle[0]:
                                    #Vehicle FULL:
                                    consumption_temp[(keys,path_index,commodities[c],days[d],node_index)]=0
                                else:
                                    amt_consumed=0
                                    #Find difference
                                    difference=values_vehicle[0]-supply_temp
                                    if difference>0:
                                        if difference>capacity_dict[(node,commodities[c])]:
                                            supply_temp+=min(capacity_dict[(node,commodities[c])],max(rem_demand-supply_temp,0))
                                            amt_consumed=min(capacity_dict[(node,commodities[c])],max(rem_demand-supply_temp,0))
                                        elif difference<=capacity_dict[(node,commodities[c])]:
                                            supply_temp+=min(difference,max(rem_demand-supply_temp,0))
                                            amt_consumed=min(difference,max(rem_demand-supply_temp,0))
                                        # elif difference==capacity_dict[(node,commodities[c])]:
                                        #     supply_temp+=difference
                                        #     amt_consumed=difference
                                        consumption_temp[((keys,path_index,commodities[c],days[d],node_index))]=amt_consumed
                                    else:
                                        print("OMG!!! Somethign has seriosly gone wrong")

                                    
                            elif node in depot_nodes and node_index==0:

                            
                                
                                if values_vehicle[0]<=capacity_dict[(node,commodities[c])]:
                                    supply_temp=min(values_vehicle[0],rem_demand)
                                    consumption_temp[(keys,path_index,commodities[c],days[d],node_index)]=min(values_vehicle[0],rem_demand)
                                else:
                                    supply_temp=min(capacity_dict[(node,commodities[c])],rem_demand)
                                    consumption_temp[(keys,path_index,commodities[c],days[d],node_index)]=min(capacity_dict[(node,commodities[c])],rem_demand)

                            elif node in customer_nodes:
                                if supply_temp >= demand_dict[(node,commodities[c],days[d])]:
                                    supply_temp-=demand_dict[(node,commodities[c],days[d])]
                                else:
                                    checker=False
                        else:
                            continue
                                
                    if checker==True:
                        feasible_path_dict[(keys,commodities[c],days[d])][path_index]=path_dict[(keys,commodities[c],days[d])][path_index]
                        feasible_path_dict[(keys,commodities[c],days[d])][path_index]['ConsumptionTracker']=consumption_temp
                        total_demand_to_be_served_by_path=0
                        for serve_node in feasible_path_dict[(keys,commodities[c],days[d])][path_index]['CustomerSet']:
                            total_demand_to_be_served_by_path+=demand_dict[(serve_node,commodities[c],days[d])]
                        feasible_path_dict[(keys,commodities[c],days[d])][path_index]['TotalDemandServed']=total_demand_to_be_served_by_path

                else:
                    #print("----",path_dict[keys,commodities[c]][path_index])
                    if path_dict[(keys,commodities[c],days[d])][path_index]['Path'][1] in customer_nodes and path_dict[(keys,commodities[c],days[d])][path_index]['Path'][0] in depot_nodes and values_vehicle[0]>=demand_dict[(path_dict[(keys,commodities[c],days[d])][path_index]['Path'][1],commodities[c],days[d])] and capacity_dict[(path_dict[(keys,commodities[c],days[d])][path_index]['Path'][0],commodities[c])]>=demand_dict[(path_dict[(keys,commodities[c])][path_index]['Path'][1],commodities[c],days[d])]:
                        feasible_path_dict[(keys,commodities[c],days[d])][path_index]=path_dict[(keys,commodities[c],days[d])][path_index]
                        feasible_path_dict[(keys,commodities[c],days[d])][path_index]['ConsumptionTracker']={(keys,path_index,commodities[c],days[d],node_index):demand_dict[(path_dict[(keys,commodities[c])][path_index]['Path'][1],commodities[c],days[d])]}
                        total_demand_to_be_served_by_path=0
                        for serve_node in feasible_path_dict[(keys,commodities[c],days[d])][path_index]['CustomerSet']:
                            total_demand_to_be_served_by_path+=demand_dict[(serve_node,commodities[c],days[d])]
                        feasible_path_dict[(keys,commodities[c],days[d])][path_index]['TotalDemandServed']=total_demand_to_be_served_by_path
                        #print("HELLO")
            # else:
            #     print(Counter(temp)[False])


In [50]:
feasible_vehicle_dict={}
for cust_node in customer_nodes:
    for c in range(len(commodities)):
        for d in range(len(days)):
            feasible_vehicle_dict[(cust_node,commodities[c],days[d])]=[]
            for vehicle_index,v1 in vehicle_dict.items():
                if demand_dict[(cust_node,commodities[c],days[d])]<=v1[0]:
                    feasible_vehicle_dict[(cust_node,commodities[c],days[d])].append(vehicle_index)
    



In [51]:
total_demand_to_be_satisfied=0
for cust_node in customer_nodes:
    for c in range(len(commodities)):
        for d in range(len(days)):
            total_demand_to_be_satisfied+=demand_dict[(cust_node,commodities[c],days[d])]


In [52]:


z_decision_variable_indices=[]
objective_function_values={}
for vehicle_index,v1 in vehicle_dict.items():
    for c in range(len(commodities)): 
        for d in range(len(days)):
            for path_index,v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                    z_decision_variable_indices.append((vehicle_index,path_index,commodities[c],days[d]))
                    objective_function_values[(vehicle_index,path_index,commodities[c],days[d])]= feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['Cost']-( (10**10) *feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['TotalDemandServed']) # + f
                    

from docplex.mp.model import Model

vrp_model = Model('Heterogenous Fleet MDMT-VRP')
z=vrp_model.binary_var_dict(z_decision_variable_indices,name='z')


obj_fn=0
for vehicle_index,v1 in vehicle_dict.items():
    for c in range(len(commodities)):
        for d in range(len(days)):
            for path_index,v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                obj_fn+=objective_function_values[(vehicle_index,path_index,commodities[c],days[d])]*z[(vehicle_index,path_index,commodities[c],days[d])]

obj_fn+=total_demand_to_be_satisfied * (10**10)


vrp_model.set_objective("min",obj_fn)

for j in customer_nodes:
    for c in range(len(commodities)):
        for d in range(len(days)):
            c_once=0
            for vehicle_index,v1 in vehicle_dict.items():
                if vehicle_index in feasible_vehicle_dict[(j,commodities[c],days[d])]:         
                    for path_index,v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                        if j in feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['CustomerSet']:
                            c_once+=z[(vehicle_index,path_index,commodities[c],days[d])]
                else:
                    continue
            if c_once:
                #print("BRO")
                vrp_model.add_constraint(c_once<=1,'cvo') #customer visit once(cvo)



for i in depot_nodes:
    for c in range(len(commodities)):
        depot_cap_constraint=0
        for vehicle_index,v1 in vehicle_dict.items():
            for d in range(len(days)):
                for path_index,v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                    for cons_key,cons_value in feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['ConsumptionTracker'].items():
                        if feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['Path'][cons_key[-1]] == i:
                            depot_cap_constraint+=(cons_value * z[(vehicle_index,path_index,commodities[c],days[d])])

                    # if feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['Start'] == i:
                    #     depot_cap_constraint+=(feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['ConsumptionTracker']*z[(vehicle_index,path_index,commodities[c],days[d])]) 
        if depot_cap_constraint:
            #print("HELLO",i,commodities[c])
            vrp_model.add_constraint(depot_cap_constraint <= capacity_dict[(i,commodities[c])], 'dcc') #depot capacity constraint(dcc)

# #------- --------------------------------!!!!!!!!!!!!!!!!!!!!!!!!!!

for i in depot_nodes:
    for vehicle_index, v1 in vehicle_dict.items():
        for c in range(len(commodities)):
            for d in range(len(days)):
                lhs=0
                for j in depot_nodes:
                    for path_index, v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                        if feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['Start']==i and feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['End']==j:
                            lhs+=z[(vehicle_index,path_index,commodities[c],days[d])]
                
                rhs=0
                for j in depot_nodes:
                    for path_index, v2 in feasible_path_dict[(vehicle_index,commodities[c],days[d])].items():
                        if feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['Start']==j and feasible_path_dict[(vehicle_index,commodities[c],days[d])][path_index]['End']==i:
                            rhs+=z[(vehicle_index,path_index,commodities[c],days[d])]

                #print("hhh",i,vehicle_index,commodities)
                vrp_model.add_constraint(lhs==rhs, 'dbc') #depot balance constraint(dbc)
            


sol=vrp_model.solve()




sol.get_objective_value()




1.2187990001740152e+16

In [53]:
sol_values=sol.get_value_dict(z)
for k,v in sol_values.items():
    if v==1:
        print(True,k)


True ('V4', 2078, 'P1', 'D1')
True ('V4', 1208, 'P1', 'D2')
True ('V4', 2748, 'P1', 'D3')
True ('V4', 2285, 'P1', 'D4')
True ('V4', 2285, 'P1', 'D10')
True ('V4', 1126, 'P1', 'D12')
True ('V4', 2772, 'P1', 'D13')
True ('V4', 1368, 'P1', 'D14')
True ('V4', 1126, 'P1', 'D15')
True ('V4', 2285, 'P2', 'D1')
True ('V4', 2285, 'P2', 'D2')
True ('V4', 2285, 'P2', 'D3')
True ('V4', 2285, 'P2', 'D4')
True ('V4', 2078, 'P2', 'D5')
True ('V4', 2285, 'P2', 'D6')
True ('V4', 2285, 'P2', 'D7')
True ('V4', 2078, 'P2', 'D8')
True ('V4', 2285, 'P2', 'D9')
True ('V4', 2285, 'P2', 'D10')
True ('V4', 2285, 'P2', 'D11')
True ('V4', 2285, 'P2', 'D12')
True ('V4', 2285, 'P2', 'D13')
True ('V4', 2285, 'P2', 'D14')
True ('V4', 2285, 'P2', 'D15')
True ('V4', 2078, 'P3', 'D1')
True ('V4', 2748, 'P3', 'D2')
True ('V4', 2078, 'P3', 'D3')
True ('V4', 2748, 'P3', 'D4')
True ('V4', 2748, 'P3', 'D6')
True ('V4', 2078, 'P3', 'D7')
True ('V4', 2078, 'P3', 'D8')
True ('V4', 2078, 'P3', 'D9')
True ('V4', 2748, 'P3', 'D10'

In [54]:
sol_values=sol.get_value_dict(z)
sol_paths=[]
print("-----> FINAL OBJECTIVE VALUE : ", sol.get_objective_value())
print("\n\n\n")
demand_satisfied_by_model=0
for k,v in sol_values.items():
    if v==1:
        print("**********************************************************")
        print("VESSEL TYPE   : ",k[0])
        print("ROUTE NO      : ",k[1])
        print("PRODUCT TYPE  : ",k[2])
        print("START REFINERY: ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['Start'])
        print("END REFINERY  : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['End'])
        print("ROUTE         : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['Path'])
        print("ROUTECOST     : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['Cost'])
        print("DAY           : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['Day'])
        print("CONSUMPTION   : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['ConsumptionTracker'])
        print("CUSTOMER SET  : ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['CustomerSet'])
        print("TOTALDEMANDSER: ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['TotalDemandServed'])
        print("EACHCUSTOMERDE: ",feasible_path_dict[(k[0],k[2],k[3])][k[1]]['EachCustomerDemand'])
        demand_satisfied_by_model+=feasible_path_dict[(k[0],k[2],k[3])][k[1]]['TotalDemandServed']
        sol_paths.append((k,feasible_path_dict[(k[0],k[2],k[3])][k[1]]))
        print("**********************************************************\n\n")


            

-----> FINAL OBJECTIVE VALUE :  1.2187990001740152e+16




**********************************************************
VESSEL TYPE   :  V4
ROUTE NO      :  2078
PRODUCT TYPE  :  P1
START REFINERY:  A
END REFINERY  :  A
ROUTE         :  ('A', 'C', 'D', 'E', 'G', 'F', 'A')
ROUTECOST     :  36450
DAY           :  D1
CONSUMPTION   :  {('V4', 2078, 'P1', 'D1', 0): 62870}
CUSTOMER SET  :  ['E', 'G', 'D', 'F', 'C']
TOTALDEMANDSER:  62870
EACHCUSTOMERDE:  [0, 14780, 11637, 12353, 13497, 10603, 0]
**********************************************************


**********************************************************
VESSEL TYPE   :  V4
ROUTE NO      :  1208
PRODUCT TYPE  :  P1
START REFINERY:  B
END REFINERY  :  B
ROUTE         :  ('B', 'G', 'E', 'B')
ROUTECOST     :  33750
DAY           :  D2
CONSUMPTION   :  {('V4', 1208, 'P1', 'D2', 0): 22102}
CUSTOMER SET  :  ['E', 'G']
TOTALDEMANDSER:  22102
EACHCUSTOMERDE:  [0, 10142, 11960, 0]
**********************************************************


**

In [55]:

total_routing_cost=0
for path in sol_paths:
    total_routing_cost+=path[1]['Cost']
vehicle_set_final_solution=[]
ct_refills=0
for k,v in sol_values.items():
    if v==1:
        if k[0] not in vehicle_set_final_solution:
            vehicle_set_final_solution.append(k[0])
        cur_consumption_list=list(feasible_path_dict[(k[0],k[2],k[3])][k[1]]['ConsumptionTracker'].values())
        if len(cur_consumption_list)>1:
            for x in range(1,len(cur_consumption_list)):
                if cur_consumption_list[x]>0:
                    ct_refills+=1

In [56]:
print("No. of Paths              : ",len(global_paths_set))
print("No  of Decision Variables : ", len(z))
print("No. of Routes Selected    : ",len(sol_paths))
print("Total Routing Cost        : ",total_routing_cost)
print("Demand Satisfied by Model : ",demand_satisfied_by_model)
print("Total Demand to be Sat    : ",total_demand_to_be_satisfied)
print("Vehicle Set               : ",vehicle_set_final_solution)
print("No of Refills             : ",ct_refills)

No. of Paths              :  3912
No  of Decision Variables :  1261620
No. of Routes Selected    :  44
Total Routing Cost        :  1703700
Demand Satisfied by Model :  2447009
Total Demand to be Sat    :  3724991
Vehicle Set               :  ['V4']
No of Refills             :  0
